Columns:
['travel_mode', 'linked_trip_id', 'trip_id', 'tour_id', 'device_id', 'geohash7_orig', 'geohash7_dest', 'local_datetime_start', 'local_datetime_end', 'route_taken', 'observed_link', 'observed_time', 'network_distance', 'route_distance', 'route_speed', 'geohash7_home', 'access_stop', 'access_stop_id', 'egress_stop', 'egress_stop_id', 'trip_purpose', 'trip_weight', 'flag_enter', 'flag_exit']


In [2]:
import os
import json
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

# =========================
# Paths
# =========================
DELIVERY_ROOT = r"C:\Users\rli04\Villanova University\Complete-trip-coordinate - Documents\General\Salt_Lake\delivery"
CENSUS_FILE = r"C:\Github\Complete-Trip-Data-Explorer\data\census_track\CensusTracts2020_6_counties.geojson"
OUT_JSON = r"./od_monthly_linked_unlinked.json"

# =========================
# Load census tracts
# =========================
tracts = gpd.read_file(CENSUS_FILE).to_crs(epsg=4326)
TRACT_COL = "GEOID"

# =========================
# Pure python geohash decode
# =========================
_base32 = "0123456789bcdefghjkmnpqrstuvwxyz"
_base32_map = {c: i for i, c in enumerate(_base32)}

def decode_geohash(gh):
    lat_interval = [-90.0, 90.0]
    lon_interval = [-180.0, 180.0]
    is_even = True
    for c in gh:
        cd = _base32_map[c]
        for mask in (16, 8, 4, 2, 1):
            if is_even:
                mid = (lon_interval[0] + lon_interval[1]) / 2
                lon_interval[0 if cd & mask else 1] = mid
            else:
                mid = (lat_interval[0] + lat_interval[1]) / 2
                lat_interval[0 if cd & mask else 1] = mid
            is_even = not is_even
    return (lon_interval[0] + lon_interval[1]) / 2, (lat_interval[0] + lat_interval[1]) / 2

# =========================
# Main container
# =========================
all_months = []

# =========================
# Loop months
# =========================
for folder in tqdm(os.listdir(DELIVERY_ROOT)):
    if not folder.startswith("Salt_Lake-"):
        continue

    month_dir = os.path.join(DELIVERY_ROOT, folder)

    dfs = [
        pd.read_parquet(os.path.join(month_dir, f))
        for f in os.listdir(month_dir)
        if f.endswith(".parquet")
    ]
    if not dfs:
        continue

    df = pd.concat(dfs, ignore_index=True)

    # =========================
    # Basic cleaning
    # =========================
    df = df[
        df["geohash7_orig"].notna() &
        df["geohash7_dest"].notna()
    ].copy()

    df["local_datetime_start"] = pd.to_datetime(df["local_datetime_start"])
    df["month"] = df["local_datetime_start"].dt.to_period("M").astype(str)
    df["trip_weight"] = df["trip_weight"].fillna(1.0)

    # =========================
    # Decode geohash → lon/lat
    # =========================
    df[["o_lon", "o_lat"]] = df["geohash7_orig"].apply(
        lambda x: pd.Series(decode_geohash(x))
    )
    df[["d_lon", "d_lat"]] = df["geohash7_dest"].apply(
        lambda x: pd.Series(decode_geohash(x))
    )

    # =========================
    # Spatial join (origin)
    # =========================
    o_gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.o_lon, df.o_lat),
        crs="EPSG:4326"
    )
    o_join = gpd.sjoin(
        o_gdf,
        tracts[[TRACT_COL, "geometry"]],
        how="left",
        predicate="intersects"
    ).rename(columns={TRACT_COL: "origin_tract"})

    # =========================
    # Spatial join (destination)
    # =========================
    d_gdf = gpd.GeoDataFrame(
        o_join,
        geometry=gpd.points_from_xy(o_join.d_lon, o_join.d_lat),
        crs="EPSG:4326"
    )
    d_join = gpd.sjoin(
        d_gdf,
        tracts[[TRACT_COL, "geometry"]],
        how="left",
        predicate="intersects"
    ).rename(columns={TRACT_COL: "destination_tract"})

    # =========================
    # -------- UNLINKED OD --------
    # =========================
    unlinked_od = (
        d_join
        .groupby(
            [
                "month",
                "origin_tract",
                "destination_tract",
                "travel_mode",
                "o_lon", "o_lat",
                "d_lon", "d_lat"
            ],
            as_index=False
        )
        .agg(
            unlinked_count=("trip_id", "count"),
            unlinked_weighted_flow=("trip_weight", "sum")
        )
    )

    # =========================
    # -------- LINKED OD (CORRECT) --------
    # =========================
    # 关键：先按时间排序，再 collapse
    d_join = d_join.sort_values(["linked_trip_id", "local_datetime_start"])

    linked_base = (
        d_join
        .groupby("linked_trip_id", as_index=False)
        .agg(
            month=("month", "first"),
            travel_mode=("travel_mode", "first"),

            origin_tract=("origin_tract", "first"),
            o_lon=("o_lon", "first"),
            o_lat=("o_lat", "first"),

            destination_tract=("destination_tract", "last"),
            d_lon=("d_lon", "last"),
            d_lat=("d_lat", "last"),

            linked_weight=("trip_weight", "first")
        )
    )

    linked_od = (
        linked_base
        .groupby(
            [
                "month",
                "origin_tract",
                "destination_tract",
                "travel_mode",
                "o_lon", "o_lat",
                "d_lon", "d_lat"
            ],
            as_index=False
        )
        .agg(
            linked_count=("linked_trip_id", "count"),
            linked_weighted_flow=("linked_weight", "sum")
        )
    )

    # =========================
    # Merge linked + unlinked
    # =========================
    od = pd.merge(
        unlinked_od,
        linked_od,
        on=[
            "month",
            "origin_tract",
            "destination_tract",
            "travel_mode",
            "o_lon", "o_lat",
            "d_lon", "d_lat"
        ],
        how="outer"
    )

    all_months.append(od)

# =========================
# Final output
# =========================
final_df = pd.concat(all_months, ignore_index=True)

# JSON safety
final_df = final_df.fillna(0)
final_df[["origin_tract", "destination_tract"]] = (
    final_df[["origin_tract", "destination_tract"]].replace(0, "UNKNOWN")
)

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(final_df.to_dict(orient="records"), f, indent=2)

print(f"Saved {len(final_df)} OD records to {OUT_JSON}")


  4%|▍         | 1/26 [02:46<1:09:28, 166.73s/it]


KeyError: "['GEOID'] not in index"

In [7]:
import os
import json
import pandas as pd
import geopandas as gpd
import pygeohash as pgh
from tqdm import tqdm

# =========================
# Paths
# =========================
DELIVERY_ROOT = r"C:\Users\rli04\Villanova University\Complete-trip-coordinate - Documents\General\Salt_Lake\delivery"
CENSUS_FILE = r"C:\Github\Complete-Trip-Data-Explorer\data\census_track\CensusTracts2020_6_counties.geojson"

OUT_OD_JSON = r"./od_monthly_linked_unlinked.json"
OUT_TRACT_CENTROID_JSON = r"./tract_centroids.json"

# =========================
# Load census tracts
# =========================
tracts = gpd.read_file(CENSUS_FILE).to_crs(epsg=4326)
TRACT_COL = "GEOID20"

# =========================
# Precompute tract centroids (ONCE)
# =========================
tracts["centroid"] = tracts.geometry.centroid

tract_centroids = {
    str(row[TRACT_COL]): {
        "lat": row.centroid.y,
        "lon": row.centroid.x
    }
    for _, row in tracts.iterrows()
}

with open(OUT_TRACT_CENTROID_JSON, "w", encoding="utf-8") as f:
    json.dump(tract_centroids, f, indent=2)

print(f"Saved tract centroids → {OUT_TRACT_CENTROID_JSON}")

# =========================
# Build GLOBAL geohash → tract lookup
# =========================
print("Building geohash → tract lookup (ONE TIME)...")

all_geohash = set()

for folder in os.listdir(DELIVERY_ROOT):
    if not folder.startswith("Salt_Lake-"):
        continue
    month_dir = os.path.join(DELIVERY_ROOT, folder)
    for f in os.listdir(month_dir):
        if f.endswith(".parquet"):
            tmp = pd.read_parquet(
                os.path.join(month_dir, f),
                columns=["geohash7_orig", "geohash7_dest"]
            )
            all_geohash.update(tmp["geohash7_orig"].dropna().unique())
            all_geohash.update(tmp["geohash7_dest"].dropna().unique())

all_geohash = list(all_geohash)
print(f"Unique geohash count: {len(all_geohash)}")

# decode once
gh_df = pd.DataFrame({
    "geohash": all_geohash,
    "lat": [pgh.decode(g)[0] for g in all_geohash],
    "lon": [pgh.decode(g)[1] for g in all_geohash],
})

gh_gdf = gpd.GeoDataFrame(
    gh_df,
    geometry=gpd.points_from_xy(gh_df.lon, gh_df.lat),
    crs="EPSG:4326"
)

# spatial join ONCE
gh_join = gpd.sjoin(
    gh_gdf,
    tracts,
    how="left",
    predicate="intersects"
).drop(columns=[c for c in gh_gdf.columns if c.startswith("index_")])

geohash2tract = dict(
    zip(gh_join["geohash"], gh_join[TRACT_COL])
)

print("Geohash → tract lookup built.")

# =========================
# Main OD container
# =========================
all_months = []

# =========================
# Loop months (NO sjoin here)
# =========================
for folder in tqdm(os.listdir(DELIVERY_ROOT), desc="Processing months"):
    if not folder.startswith("Salt_Lake-"):
        continue

    month_dir = os.path.join(DELIVERY_ROOT, folder)

    dfs = [
        pd.read_parquet(os.path.join(month_dir, f))
        for f in os.listdir(month_dir)
        if f.endswith(".parquet")
    ]
    if not dfs:
        continue

    df = pd.concat(dfs, ignore_index=True)

    # =========================
    # Basic cleaning
    # =========================
    df = df[
        df["geohash7_orig"].notna() &
        df["geohash7_dest"].notna()
    ].copy()

    df["local_datetime_start"] = pd.to_datetime(df["local_datetime_start"])
    df["month"] = df["local_datetime_start"].dt.to_period("M").astype(str)
    df["trip_weight"] = df["trip_weight"].fillna(1.0)

    # =========================
    # Map geohash → tract (FAST)
    # =========================
    df["origin_tract"] = df["geohash7_orig"].map(geohash2tract)
    df["destination_tract"] = df["geohash7_dest"].map(geohash2tract)

    df = df[
        df["origin_tract"].notna() &
        df["destination_tract"].notna()
    ]

    # =========================
    # -------- UNLINKED OD --------
    # =========================
    unlinked_od = (
        df
        .groupby(
            ["month", "origin_tract", "destination_tract", "travel_mode"],
            as_index=False
        )
        .agg(
            unlinked_count=("trip_id", "count"),
            unlinked_weighted_flow=("trip_weight", "sum")
        )
    )

    # =========================
    # -------- LINKED OD --------
    # =========================
    df = df.sort_values(["linked_trip_id", "local_datetime_start"])

    linked_base = (
        df
        .groupby("linked_trip_id", as_index=False)
        .agg(
            month=("month", "first"),
            travel_mode=("travel_mode", "first"),
            origin_tract=("origin_tract", "first"),
            destination_tract=("destination_tract", "last"),
            linked_weight=("trip_weight", "first")
        )
    )

    linked_od = (
        linked_base
        .groupby(
            ["month", "origin_tract", "destination_tract", "travel_mode"],
            as_index=False
        )
        .agg(
            linked_count=("linked_trip_id", "count"),
            linked_weighted_flow=("linked_weight", "sum")
        )
    )

    # =========================
    # Merge linked + unlinked
    # =========================
    od = pd.merge(
        unlinked_od,
        linked_od,
        on=["month", "origin_tract", "destination_tract", "travel_mode"],
        how="outer"
    )

    all_months.append(od)

# =========================
# Final output
# =========================
final_df = pd.concat(all_months, ignore_index=True).fillna(0)

final_df[["origin_tract", "destination_tract"]] = (
    final_df[["origin_tract", "destination_tract"]].astype(str)
)

with open(OUT_OD_JSON, "w", encoding="utf-8") as f:
    json.dump(final_df.to_dict(orient="records"), f, indent=2)

print(f"Saved tract-level OD → {OUT_OD_JSON}")


C:\Users\rli04\AppData\Local\Temp\ipykernel_13432\487544621.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts["centroid"] = tracts.geometry.centroid


Saved tract centroids → ./tract_centroids.json
Building geohash → tract lookup (ONE TIME)...
Unique geohash count: 191678
Geohash → tract lookup built.


Processing months: 100%|██████████| 26/26 [12:10<00:00, 28.11s/it]


Saved tract-level OD → ./od_monthly_linked_unlinked.json


In [10]:
# =========================
# Build DASHBOARD-level TOP-K OD (WITH COORDINATES) — FINAL
# =========================

TOP_K = 20   # 推荐 20；10 会太稀疏

print(f"Building TOP-{TOP_K} OD for dashboard (excluding self-loops)...")

# -------------------------------------------------
# 1️⃣ 计算总 flow（排序指标）
# -------------------------------------------------
final_df = final_df.copy()

final_df["total_flow"] = (
    final_df["linked_weighted_flow"].fillna(0) +
    final_df["unlinked_weighted_flow"].fillna(0)
)

# -------------------------------------------------
# 2️⃣ 过滤：去掉同 O 同 D（自环）
# -------------------------------------------------
before_cnt = len(final_df)

final_df = final_df[
    final_df["origin_tract"] != final_df["destination_tract"]
]

print(f"Removed self-loop OD rows: {before_cnt - len(final_df)}")

# -------------------------------------------------
# 3️⃣ 按 month 选 TOP-K
# -------------------------------------------------
topk_df = (
    final_df
    .sort_values("total_flow", ascending=False)
    .groupby("month", group_keys=False)
    .head(TOP_K)
)

print(f"Dashboard OD rows after TOP-{TOP_K}: {len(topk_df)}")

# -------------------------------------------------
# 4️⃣ 加载 tract centroids（一次）
# -------------------------------------------------
with open(OUT_TRACT_CENTROID_JSON, "r", encoding="utf-8") as f:
    tract_centroids = json.load(f)

centroid_df = (
    pd.DataFrame
    .from_dict(tract_centroids, orient="index")
    .rename(columns={"lat": "lat", "lon": "lon"})
)

centroid_df.index.name = "tract"
centroid_df.reset_index(inplace=True)
centroid_df["tract"] = centroid_df["tract"].astype(str)

# -------------------------------------------------
# 5️⃣ 向量化 join 坐标（快 & 稳）
# -------------------------------------------------
topk_df["origin_tract"] = topk_df["origin_tract"].astype(str)
topk_df["destination_tract"] = topk_df["destination_tract"].astype(str)

topk_df = (
    topk_df
    .merge(
        centroid_df.rename(columns={
            "tract": "origin_tract",
            "lat": "o_lat",
            "lon": "o_lon"
        }),
        on="origin_tract",
        how="left"
    )
    .merge(
        centroid_df.rename(columns={
            "tract": "destination_tract",
            "lat": "d_lat",
            "lon": "d_lon"
        }),
        on="destination_tract",
        how="left"
    )
)

# -------------------------------------------------
# 6️⃣ 严格清洗非法坐标（关键）
# -------------------------------------------------
before_geo = len(topk_df)

topk_df = topk_df.dropna(
    subset=["o_lat", "o_lon", "d_lat", "d_lon"]
)

# 防 NaN / inf / 非数值
for c in ["o_lat", "o_lon", "d_lat", "d_lon"]:
    topk_df = topk_df[pd.to_numeric(topk_df[c], errors="coerce").notna()]

print(f"Removed OD rows with invalid centroid: {before_geo - len(topk_df)}")

# -------------------------------------------------
# 7️⃣ 只保留 Dashboard 必需字段
# -------------------------------------------------
dashboard_cols = [
    "month",
    "origin_tract",
    "destination_tract",
    "o_lat", "o_lon",
    "d_lat", "d_lon",
    "travel_mode",
    "linked_count",
    "unlinked_count",
    "linked_weighted_flow",
    "unlinked_weighted_flow",
    "total_flow"
]

topk_df = topk_df[dashboard_cols]

# -------------------------------------------------
# 8️⃣ 非坐标字段 fillna（坐标严禁填）
# -------------------------------------------------
count_cols = [
    "linked_count",
    "unlinked_count",
    "linked_weighted_flow",
    "unlinked_weighted_flow",
    "total_flow"
]

topk_df[count_cols] = topk_df[count_cols].fillna(0)

# -------------------------------------------------
# 9️⃣ 输出 Dashboard JSON
# -------------------------------------------------
OUT_DASHBOARD_JSON = "./od_dashboard_topk.json"

with open(OUT_DASHBOARD_JSON, "w", encoding="utf-8") as f:
    json.dump(
        topk_df.to_dict(orient="records"),
        f,
        indent=2
    )

print(
    f"Saved dashboard TOP-{TOP_K} OD with coordinates "
    f"(no self-loops, safe) → {OUT_DASHBOARD_JSON}"
)


Building TOP-20 OD for dashboard (excluding self-loops)...
Removed self-loop OD rows: 0
Dashboard OD rows after TOP-20: 240
Removed OD rows with invalid centroid: 0
Saved dashboard TOP-20 OD with coordinates (no self-loops, safe) → ./od_dashboard_topk.json
